In [7]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf spark-3.2.3-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/spark-3.2.3-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.17.jar

--2022-12-17 21:14:53--  https://jdbc.postgresql.org/download/postgresql-42.2.17.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1004734 (981K) [application/java-archive]
Saving to: ‘postgresql-42.2.17.jar’

postgresql-42.2.17. 100%[===================>] 981.19K  1.18MB/s    in 0.8s    

2022-12-17 21:14:55 (1.18 MB/s) - ‘postgresql-42.2.17.jar’ saved [1004734/1004734]



In [9]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.17.jar").getOrCreate()

In [25]:
import pandas as pd
import io
from io import StringIO

# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://module17.9.1-bucket.s3.amazonaws.com/user_data.csv"
spark.sparkContext.addFile(url)
from google.colab import files
uploaded = files.upload()

Saving user_data.csv to user_data (1).csv


In [28]:
# Show DataFrame
user_data_df = pd.read_csv(io.BytesIO(uploaded['user_data.csv']))
user_data_df.head()

,id,first_name,last_name,active_user,street_address,state,username
0,1,Cletus,Lithcow,False,78309 Riverside Way,Virginia,ibearham0
1,2,Caz,Felgat,False,83 Hazelcrest Place,Alabama,wwaller1
2,3,Kerri,Crowson,False,112 Eliot Pass,North Carolina,ichesnut2
3,4,Freddie,Caghy,False,15 Merchant Way,New York,tsnarr3
4,5,Sadella,Deuss,False,079 Acker Avenue,Tennessee,fwherrit4


In [29]:
uploaded = files.upload()

Saving user_payment.csv to user_payment (1).csv


In [31]:
url ="https://module17.9.1-bucket.s3.amazonaws.com/user_payment.csv"
spark.sparkContext.addFile(url)
user_payment_df = pd.read_csv(io.BytesIO(uploaded['user_payment.csv']))
user_payment_df.head()

,billing_id,username,cc_encrypted
0,1,ibearham0,a799fcafe47d7fb19bfb02cd83855fdfc34b9f87
1,2,wwaller1,a799fcafe47d7fb19bfb02cd83855fdfc34b9f87
2,3,ichesnut2,a799fcafe47d7fb19bfb02cd83855fdfc34b9f87
3,4,tsnarr3,a799fcafe47d7fb19bfb02cd83855fdfc34b9f87
4,5,fwherrit4,a799fcafe47d7fb19bfb02cd83855fdfc34b9f87


In [33]:
# Join the two DataFrame
joined_df = user_data_df.merge(user_payment_df, on="username", how="inner")
joined_df.head()

,id,first_name,last_name,active_user,street_address,state,username,billing_id,cc_encrypted
0,1,Cletus,Lithcow,False,78309 Riverside Way,Virginia,ibearham0,1,a799fcafe47d7fb19bfb02cd83855fdfc34b9f87
1,2,Caz,Felgat,False,83 Hazelcrest Place,Alabama,wwaller1,2,a799fcafe47d7fb19bfb02cd83855fdfc34b9f87
2,3,Kerri,Crowson,False,112 Eliot Pass,North Carolina,ichesnut2,3,a799fcafe47d7fb19bfb02cd83855fdfc34b9f87
3,4,Freddie,Caghy,False,15 Merchant Way,New York,tsnarr3,4,a799fcafe47d7fb19bfb02cd83855fdfc34b9f87
4,5,Sadella,Deuss,False,079 Acker Avenue,Tennessee,fwherrit4,5,a799fcafe47d7fb19bfb02cd83855fdfc34b9f87


In [34]:
# Drop null values
dropna_df = joined_df.dropna()
dropna_df.head()

,id,first_name,last_name,active_user,street_address,state,username,billing_id,cc_encrypted
0,1,Cletus,Lithcow,False,78309 Riverside Way,Virginia,ibearham0,1,a799fcafe47d7fb19bfb02cd83855fdfc34b9f87
1,2,Caz,Felgat,False,83 Hazelcrest Place,Alabama,wwaller1,2,a799fcafe47d7fb19bfb02cd83855fdfc34b9f87
2,3,Kerri,Crowson,False,112 Eliot Pass,North Carolina,ichesnut2,3,a799fcafe47d7fb19bfb02cd83855fdfc34b9f87
3,4,Freddie,Caghy,False,15 Merchant Way,New York,tsnarr3,4,a799fcafe47d7fb19bfb02cd83855fdfc34b9f87
4,5,Sadella,Deuss,False,079 Acker Avenue,Tennessee,fwherrit4,5,a799fcafe47d7fb19bfb02cd83855fdfc34b9f87


In [56]:
new_df = dropna_df.loc(["active_user"] == True)
new_df.head()

AttributeError: ignored

In [51]:
# Load in a sql function to use columns
from pyspark.sql.functions import col

# Filter for only columns with active users
cleaned_df = dropna_df.filter(col("active_user") ==True)
cleaned_df.head()

TypeError: ignored